## Scala 中的 for 推导式
摘抄自《scala程序设计（第2版）》————3.6 Scala 中的 for 推导式

Scala 为 for 循环这一常见的控制结构提供了非常多的特性,这些for 循环的特性被称为 for 推导式(for comprehension)或 for 表达式(for expression)。

事实上,推导式一词起源于函数式编程。它表达了这样一个理念:我们遍历一个或多个集合,对集合中的元素进行“推导”,并从中计算出新的事物,新推导出的事物往往是另一个集合。

**注意 for 表达式并不局限于使用列表。任何数据类型只要支持 withFilter，map，和 flatMap 操作（不同数据类型可能支持不同的操作）都可以用来做序列推导。**

### for 循环
让我们从一个基本的 for 表达式开始:
```scala
// src/main/scala/progscala2/rounding/basic-for.sc

val dogBreeds = List("Doberman", "Yorkshire Terrier", "Dachshund",
                     "Scottish Terrier", "Great Dane", "Portuguese Water Dog")

for (breed <- dogBreeds)
  println(breed)
```
你可能已经猜到了,这段代码的意思是:“基于列表 dogBreeds 中的每一个元素,创建临时变量 breed , breed 的值与元素值相同,之后打印 breed 。”代码输出如下:
```
Doberman
Yorkshire Terrier
Dachshund
Scottish Terrier
Great Dane
Portuguese Water Dog
```
这种形式不返回任何值,因此它只会执行一些会带来副作用的操作。这类 for 推导式有时候也被称为 for 循环,这与 Java 中的 for 循环较为类似。

### 生成器表达式
像 breed <- dogBreeds 这样的表达式也被称为生成器表达式(generator expression),生成器表达式之所以叫这个名字,是因为该表达式会基于集合生成单独的数值。左箭头操作符(<-) 用于对像列表这样的集合进行遍历。

我们还可以使用生成器表达式对某些区间进行访问,以这种方式编写出的 for 循环更加自然。
```scala
// src/main/scala/progscala2/rounding/generator.sc

for (i <- 1 to 10) println(i)
```

### 保护式 : 筛选元素
怎样才能获得更细的操作粒度呢? 我们可以加入 if 表达式,来筛选出我们希望保留的元素。这些表达式也被称为保护式(guard)。 为了能够从犬种列表中挑选中 犬,我们对之前的代码进行了修改,具体如下:
```scala
// src/main/scala/progscala2/rounding/guard-for.sc

val dogBreeds = List("Doberman", "Yorkshire Terrier", "Dachshund",
                     "Scottish Terrier", "Great Dane", "Portuguese Water Dog")
for (breed <- dogBreeds
  if breed.contains("Terrier")
) println(breed)
```
输出如下:
```
Yorkshire Terrier
Scottish Terrier
```

你还可以在 for 循环中添加多个保护式:
```scala
// src/main/scala/progscala2/rounding/double-guard-for.sc

val dogBreeds = List("Doberman", "Yorkshire Terrier", "Dachshund",
                     "Scottish Terrier", "Great Dane", "Portuguese Water Dog")

for (breed <- dogBreeds
  if breed.contains("Terrier")
  if !breed.startsWith("Yorkshire")
) println(breed)

for (breed <- dogBreeds
  if breed.contains("Terrier") && !breed.startsWith("Yorkshire")
) println(breed)
```
在第二个 for 推导式中,两个 if 语句被合并为一个语句。这两个 for 推导式的输出如下所示:
```
Scottish Terrier
Scottish Terrier
```

### Yielding
假如你并不需要打印过滤后的集合,你需要编写代码对过滤后的集合进行处理,那么该怎么办呢?使用 yield 关键字便能在 for 表达式中生成新的集合。

另外,我们将转而使用大括号代替圆括号,以相似的方法把参数列表封装在大括号中时可以使得块结构的格式看起来更为直观:
```scala
// src/main/scala/progscala2/rounding/yielding-for.sc

val dogBreeds = List("Doberman", "Yorkshire Terrier", "Dachshund",
                     "Scottish Terrier", "Great Dane", "Portuguese Water Dog")
val filteredBreeds = for {
  breed <- dogBreeds
  if breed.contains("Terrier") && !breed.startsWith("Yorkshire")
} yield breed
```

每次执行 for 表达式时,过滤后的结果将生成 breed 值。随着代码的执行,这些结果值 逐 渐 积 累 起 来, 累 计 而 成 的 结 果 值 集 合 被 赋 给 了 filteredBreeds 对 象。 for-yield表 达 式 所 生 成 的 集 合 类 型 将 根 据 被 遍 历 的 集 合 类 型 推 导 而 出。 在 上 面 的 例 子 中, 由于 filteredBreeds 源 于 dogBreeds 列 表, 而 dogBreeds 类 型 为 List[String] , 因 此filteredBreeds 的类型为 List[String] 。

**for 推导式有一个不成文的约定:当 for 推导式仅包含单一表达式时使用原括号,当其包含多个表达式时使用大括号。值得注意的是,使用原括号时,早前版本的 Scala 要求表达式之间必须使用分号。**

**假如一个 for 推导式并未使用 yield ,而是执行像打印这样的具有副作用的操作,那么我们将其称为 for 循环。这是因为它的行为更像是你所熟悉的 Java 和其他语言中的 for 循环。**

### 扩展作用域与值定义
Scala 的 for 推导式还有一个有用的特征:你能够在 for 表达式中的最初部分定义值,并可以在后面的表达式中使用该值。如下所示:
```scala
// src/main/scala/progscala2/rounding/scoped-for.sc

val dogBreeds = List("Doberman", "Yorkshire Terrier", "Dachshund",
                     "Scottish Terrier", "Great Dane", "Portuguese Water Dog")
for {
  breed <- dogBreeds
  upcasedBreed = breed.toUpperCase()
} println(upcasedBreed)
```

需要注意的是,**尽管 upcasedBreed 的值不可变,但并不需要使用 val 关键字进行限定**,执行结果如下:
```
DOBERMAN
YORKSHIRE TERRIER
DACHSHUND
SCOTTISH TERRIER
GREAT DANE
PORTUGUESE WATER DOG
```

如果你想到了 Option ,那就可以用在这个示例中。正如我们之前讨论的那样, Option 是null 更好的替代方案, Option 是一类特殊形式的集合,它只包含 0 个或 1 个元素,意识到这一点对你会有帮助。我们也可以理解下面代码:
```scala
// src/main/scala/progscala2/patternmatching/scoped-option-for.sc

val dogBreeds = Seq(Some("Doberman"), None, Some("Yorkshire Terrier"), 
                    Some("Dachshund"), None, Some("Scottish Terrier"),
                    None, Some("Great Dane"), Some("Portuguese Water Dog"))

println("second pass:")
for {
  Some(breed) <- dogBreeds
  upcasedBreed = breed.toUpperCase()
} println(upcasedBreed)
```

第二个 for 推导式使用了模式匹配,这使得代码更为清新。只有当 BreedOption 是 Some 类型时,表达式 Some(breed) <- dogBreeds 才会成功执行并提取出 breed ,所有操作一步完成。 None 元素不再被处理。

什么时候使用左箭头( <- ),什么时候该使用等于号( = )呢?当你遍历某一集合或其他像Option 这样的容器并试图提取值时,你应该使用箭头。当你执行并不需要迭代的赋值操作时,你应使用等于号。 for 推导式的第一句表达式必须使用箭头符执行抽取 / 迭代操作。

在大多数语言的循环体中,你可以使用跳出循环、也可以继续进行迭代。Scala 并未提供break 和 continue 语句,不过编写地道的 Scala 代码时,你几乎不需要使用这些语句。你可以使用条件表达式或者使用递归判断循环是否应该继续。如果你能在一开始便对集合进行过滤以消除循环中的复杂条件,那就更好了 。不过,**考虑到存在对 break 功能的需求,Scala 提供了 scala.util.control.Breaks 对象,该对象可用于实现 break 功能,不过我从未用过该功能,你最好也不用它。**

## 深入学习 for 推导式
摘抄自《scala程序设计（第2版）》————第 7 章 深入学习 for 推导式

我们在 3.6 节中对 for 推导式进行了描述。学习完那一节,我们认为它们是很好用且更灵活的 for 循环,仅此而已。而实际上这只是冰山的一角,更多复杂的东西被掩藏在水面之下。这些复杂的事物有益于编写出简洁的代码,以优雅的方式解决一些设计上的难题。

在本章中,我们将深入学习被掩藏的知识以便真正理解 for 推导式。除了学到 Scala 是如何实现 for 推导式之外,我们还将学会如何在自己创建的容器类型中使用它们。

在本章的最后,我们将通过实验揭示到底有多少 Scala 容器类型使用 for 推导式解决一些常见的设计难题,例如:如何在执行一组操作时进行错误处理。最后,我们将提炼出一项知名的函数式技巧,该技巧可以用于表示重复习语。

### 内容回顾 : for 推导式组成元素
for 推导式中包含一个或多个生成器表达式,外加可选的保护表达式(guard expression,用于过滤数据)以及值定义。推导式的输出可以用于“生成”新的容器,也可以在每次遍历时执行具有副作用的代码块,如打印输出。下面的例子解释了所有这些特性。该示例移除了文本文件中所有的空行:
```scala
// src/main/scala/progscala2/forcomps/RemoveBlanks.scala
package progscala2.forcomps

object RemoveBlanks {

  /**
   * Remove blank lines from the specified input file.
   * 从指定的输入文件中移除空行。
   */
  def apply(path: String, compressWhiteSpace: Boolean = false): Seq[String] =
    for {
      line <- scala.io.Source.fromFile(path).getLines.toSeq          // <1>
      if line.matches("""^\s*$""") == false                          // <2>
      line2 = if (compressWhiteSpace) line replaceAll ("\\s+", " ")  // <3>
              else line
    } yield line2                                                    // <4>

  /**
   * Remove blank lines from the specified input files and echo the remaining
   * lines to standard output, one after the other.
   * @param args list of file paths. Prefix each with an optional "-" to
   *             compress remaining whitespace in the file.
   */
  /**
   * 从指定的输入文件中移除空行,并将其他行内容依次发送给标准输出。
   * @param 参数列表中包含了文件路径。为每一个文件路径都增加了可选的"-"前缀,
   *             并会压缩以"-"前缀开头文件中的剩余空白符。
   */
  def main(args: Array[String]): Unit = for {
    path2 <- args                                                    // <5>
    (compress, path) = if (path2 startsWith "-") (true, path2.substring(1))
                       else (false, path2)                           // <6>
    line <- apply(path, compress)
  } println(line)                                                    // <7>
}
```
➊ 使用 scala.io.Source 对象打开文件并读取文件行, getLines 返回 scala.collection.Iterator 对象。 由 于 for 推 导 式 无 法 返 回 Iterator 对象,for推导式的返回类型由初始的生成器所决定,因此我们必须将其转化成一个序列。  
➋ 使用正则表达式过滤空行。  
➌ 定义局部变量。假如未开启空白符压缩,那么局部变量将存储未变的非空行,反之则会将局部变量设置为一个新的行值,该行值已经将所有的空白符压缩为一个空格。  
➍ 由 于 我 们 使 用 yield 方 法 返 回 行 内 容, 因 此 for 推 导 式 构 造 了 apply 方 法 返 回 的Seq[String]。随后我们也将处理 apply 返回的实际容器。  
➎ main 方法使用 for 推导式处理参数列表,每个参数都会被视为待处理的文件路径。  
➏ 假如文件路径以 - 字符起始,空白符会被压缩,否则只会除去空白行。  
➐ 将所有处理后的行内容一起输出到标准输出 stdout 。  

你也许希望对该应用进行修改,添加更多的选项,例如,在每一行中增加行号,将输出写到单独的文件,计算统计值等。你该如何将参数数组中各个单独元素转变为类 Unix 风格命令行的选项呢?

我们再看看 apply 方法返回的实际容器。假如运行了 sbt 控制台,我们便能查看该容器:
```scala
> console
Welcome to Scala version 2.11.2 (Java HotSpot(TM) ...).
...

scala> val lines = forcomps.RemoveBlanks.apply(
| "src/main/scala/progscala2/forcomps/RemoveBlanks.scala")
lines: Seq[String] = Stream(
// src/main/scala/progscala2/forcomps/RemoveBlanks.scala, ?)

scala> lines.head
res1: String = // src/main/scala/progscala2/forcomps/RemoveBlanks.scala

scala> lines take 5 foreach println
// src/main/scala/progscala2/forcomps/RemoveBlanks.scala
package forcomps
object RemoveBlanks {
  /**
   * 移除指定输入文件中的空行
```

apply 方法将返回惰性 Stream 值,这点在 6.9 节中已经介绍了。当 REPL 在打印出 lines定义信息后打印行内容时, Stream.toString 方法会计算出文件流的头部内容(也就是该文件的注释行),除此之外该方法还会显示一个问号,该问号代表了文件中尚未被计算出的尾部内容。

我们可以要求获取文件的头部内容,之后获取前五行的内容,这也会迫使 Scala 计算出这些行值。由于处理的文件也许会非常庞大,如果将整个文件原封不动地载入内存会消耗大量的内存,因此此处非常适合使用 Stream 类型。不幸的是,万一需要阅读完整的大型内容集,我们就不得不把全部内容都载入内存。这是因为 Stream 会记住它所解析出的所有元素的内容。请注意由于上面出现的两个 for 推导式(分别出现在 apply 和 main 方法中)的每次迭代都不会保存状态,因此并不需要在内容中保存多于一行的数据。

事 实 上, 当 你 调 用 scala.collection.Iterator 的 toSeq 方 法 时, 会 调 用 子 类 型 scala.collection.TraversableOnce 中的默认实现并返回 Stream 类型对象。而 Iterator 的其他子类型则可能会返回一个严格型(strict)容器。

### for 推导式 : 内部机制
for 推 导 式 的 语 法 实 际 上 是 编 译 器 提 供 的 语 法 糖, 它 会 调 用 容 器 方 法 foreach 、 map 、flatMap 以及 withFilter 方法。

为 什 么 还 需 要 提 供 另 一 种 调 用 这 些 方 法 的 方 式 呢? 对 于 那 些 非 平 凡 序 列(nontrival sequence)而言,使用 for 推导式比调用相关 API 编写的代码更加易读易写。

与我们之前见到的 filter 方法一样, withFilter 方法可以对元素进行过滤。假如容器未定义 withFilter 方法,Scala 会使用 filter 方法替代(会出现编译警告)。不过,与 filter不同, withFilter 方法并不会构造输出容器。为了得到更高的效率, withFilter 会与其他方法一起执行过滤逻辑,这样能够减少一次生成新容器所带来的开销。更具体一点,withFilter 方法会限制允许传递给后续组合器的元素类型域,这些后续组合器包括 map 、flatMap 、 foreach 以及其他的 withFilter 会调用的方法。

为了能了解 for 推导式这颗语法糖里到底封装了些什么东西,我们会先执行一些非正式的比较操作,之后再探讨之前 mapping 中的详细信息。

考虑下面这个简单的 for 推导式:
```scala
// src/main/scala/progscala2/forcomps/for-foreach.sc

val states = List("Alabama", "Alaska", "Virginia", "Wyoming")

for {
  s <- states
} println(s)
// Results:
// Alabama
// Alaska
// Virginia
// Wyoming

states foreach println
// Results the same as before.
// 执行结果与之前一致。
```
注释中列出了输出结果。(从现在开始,我不会再像以前那样频繁地展示 REPL 会话信息。有时我会列出代码,并在注释中展示重要的结果。)

在推导式之后存在一个不含 yield 表达式的生成器表达式,该表达式对应了容器 foreach方法中执行的表达式。

如果我们使用 yield 操作生成容器,会发生什么呢?
```scala
// src/main/scala/progscala2/forcomps/for-map.sc

val states = List("Alabama", "Alaska", "Virginia", "Wyoming")

for {
  s <- states
} yield s.toUpperCase
// Results: List(ALABAMA, ALASKA, VIRGINIA, WYOMING)

states map (_.toUpperCase)
// Results: List(ALABAMA, ALASKA, VIRGINIA, WYOMING)
```
生成器表达式中包含了 yield 表达式,该生成器对应了一次 map 操作。那么 for 推导式是在什么时候利用 yield 操作构造出新的容器的呢?第一个生成器表达式决定了最终的结果容器类型。通过观察对应的 map 表达式的行为,你会发现这是合理的。如果将输入的 List类型修改为 Vector 类型,你会发现这将生成一个新的 Vector 容器。

如果我们定义了多个生成器,会发生什么呢?
```scala
// src/main/scala/progscala2/forcomps/for-flatmap.sc

val states = List("Alabama", "Alaska", "Virginia", "Wyoming")

for {
  s <- states
  c <- s
} yield s"$c-${c.toUpper}"
// Results: List("A-A", "l-L", "a-A", "b-B", ...)

states flatMap (_.toSeq map (c => s"$c-${c.toUpper}"))
// Results: List("A-A", "l-L", "a-A", "b-B", ...)
```
第二个生成器会遍历字符串 s 中的每一个字符。而设计的 yield 语句将返回各个字符及对应的大写字符,这两个字符通过横线分隔。

如果存在多个生成器,那么除最后一个之外,其他所有的生成器都会被转化成 flatMap 调用。最后一个生成器对应了一次 map 调用。上述代码也将产生 List 对象。你也可以尝试使用其他输入容器类型,看看输出结果是什么类型。

如果我们再添加一个保护式(guard),又会发生什么呢?
```scala
// src/main/scala/progscala2/forcomps/for-guard.sc

val states = List("Alabama", "Alaska", "Virginia", "Wyoming")

for {
  s <- states
  c <- s
  if c.isLower
} yield s"$c-${c.toUpper} "
// Results: List("l-L", "a-A", "b-B", ...)

states flatMap (_.toSeq withFilter (_.isLower) map (c => s"$c-${c.toUpper}"))
// Results: List("l-L", "a-A", "b-B", ...)
```
请注意,Scala 在最终的 map 调用之前插入了一条 withFilter 调用语句。

最后,如下所示,我们在语句中定义了一个变量:
```scala
// src/main/scala/progscala2/forcomps/for-variable.sc

val states = List("Alabama", "Alaska", "Virginia", "Wyoming")

for {
  s <- states
  c <- s
  if c.isLower
  c2 = s"$c-${c.toUpper} "
} yield c2
// Results: List("l-L", "a-A", "b-B", ...)

states flatMap (_.toSeq withFilter (_.isLower) map { c => 
  val c2 = s"$c-${c.toUpper} "
  c2
})
// Results: List("l-L", "a-A", "b-B", ...)
```

### for 推导式的转化规则
现在我们已经对 for 推导式转化成容器方法的原理有了初步的了解。下面我们将定义更加详细的细节。

首先,在像 pat <- expr 这样的生成器表达式中, pat 实际上是一个模式表达式(pattern expression),例如: (x,y) <- List((1,2),(3,4)) 。Scala 会以类似的方式对值定义语句pat2 = expr 进行处理,该语句也会被视为某一模式。

**Scala 在转化 for 推导式时,要做的第一件事便是将 pat <- expr 语句转化为下列语句**:
```scala
// pat <- expr
pat <- expr.withFilter { case pat => true; case _ => false }
```

之后,Scala 将重复执行下列转化规则,直到所有的推导表达式都被替换掉。值得一提的是,某些转化会生成新的 for 推导式,而后续的迭代则会负责对这些推导式进行转化。

如果 for 推导式中包含了一个生成器和一个 yield 表达式,那么该表达式将被转化为下列语句:
```scala
// for ( pat <- expr1 ) yield expr2
expr map { case pat => expr2 }
```

如果 for 循环中未使用 yield 语句,但执行的代码具有副作用,那么该语句将被转化为:
```scala
// for ( pat <- expr1 ) expr2
expr foreach { case pat => expr2 }
```

包含多个生成器(同时包含 yield 表达式)的 for 推导式将被转化成下列语句:
```scala
// for ( pat1 <- expr1; pat2 <- expr2; ... ) yield exprN
expr1 flatMap { case pat1 => for (pat2 <- expr2 ...) yield exprN }
```

请留意,嵌套的生成器会被转化成嵌套的 for 推导式。这些嵌套的 for 推导式会在下一次执行转化规则时被转化成方法调用。上面示例中 ( ... ) 代表了省略的表达式,这些表达式可能是其他的生成器,也可能是值定义或保护式(guard)。

包含多个生成器的 for 循环将被翻译成下列语句:
```scala
// for ( pat1 <- expr1; pat2 <- expr2; ... ) exprN
expr1 foreach { case pat1 => for (pat2 <- expr2 ...) yield exprN }
```

我们之前所见的示例中包含保护式(guard)表达式,该表达式被编写在单独的一行中。事实上,guard 以及上一行中的代码可以编写在一行中,例如: pat1 <- expr1 if guard 。后面跟着保护式的生成器会被翻译成下列语句:
```scala
// pat1 <- expr1 if guard
pat1 <- expr1 withFilter ((arg1, arg2, ...) => guard)
```
此处,变量 argN 代表了传递给 withFilter 方法的参数。对于大多数的容器而言,传入的方法中只含有一个参数。

如果生成器后面尾随着一个值定义,那么转化这个生成器的复杂度会令人惊奇。如下所示:
```scala
// pat1 <- expr1; pat2 = expr2
(pat1, pat2) <- for {        // ➊
  x1 @ pat1 <- expr1         // ➋ 
} yield {
  val x2 @ pat2 = expr2      // ➌
  (x1, x2)                   // ➍
}
```
➊ for 推导式将返回包含两个模式的 pair 对象。  
➋ x1 @ pat1 语句会将整个表达式中 pat1 所匹配的值赋给变量 x1 ,该值可能包含另一个变量的某一部分。假如 pat1 是一个不可变变量名, x1 和 pat1 的赋值将会是冗余的。  
➌ 将 pat2 值赋给 x2 。  
➍ 返回元组。  

下面的 REPL 会话中包含了 x @ pat = expr 语句的对应示例:
```scala
scala> val z @ (x, y) = (1 -> 2)
z: (Int, Int) = (1,2)
x: Int = 1
y: Int = 2
```
变量 z 的值为元组 (1,2) ,而变量 x 和变量 y 则对应了元组中各个组成部分的值。

由于很难讲清楚 for 推导式完整的转化过程,所以我们从一个具体的例子开始学起。
```scala
// src/main/scala/progscala2/forcomps/for-variable-translated.sc

val map = Map("one" -> 1, "two" -> 2)

val list1 = for {
  (key, value) <- map   // How is this line and the next translated?
  i10 = value + 10
} yield (i10)
// Result: list1: scala.collection.immutable.Iterable[Int] = List(11, 12)

// Translation:
val list2 = for {
  (i, i10) <- for {
    x1 @ (key, value) <- map
  } yield {
    val x2 @ i10 = value + 10
    (x1, x2)
  } 
} yield (i10)
// Result: list2: scala.collection.immutable.Iterable[Int] = List(11, 12)
```
请留意外围的 for {...} 中的两个表达式的转化方式。尽管在内部表达式中我们返回了(x1, x2) 对,但事实上只返回了 x2 变量(等价于 i10 变量)。另外,我们知道 Map 对象包含一组键值对元素,因此在上面的代码中,与生成器所匹配的模式类型为键值对类型,我们也使用该类型遍历 map 对象。

这便是 for 推导式完整的转化规则。你可以应用这些规则,将 for 推导式转化成针对容器的一组方法调用。你不必经常执行这样的转化,不过有时候这样做能帮助你调试问题。

我们再看一个示例,该示例应用模式匹配对一个常见的格式为 key = value 的属性文件进行解析。
```scala
// src/main/scala/progscala2/forcomps/for-patterns.sc

val ignoreRegex = """^\s*(#.*|\s*)$""".r                             // <1>
val kvRegex = """^\s*([^=]+)\s*=\s*([^#]+)\s*.*$""".r                // <2>

val properties = """
  |# Book properties
  |
  |book.name = Programming Scala, Second Edition # A comment 
  |book.authors = Dean Wampler and Alex Payne
  |book.publisher = O'Reilly
  |book.publication-year = 2014
  |""".stripMargin                                                   // <3>

val kvPairs = for {
  prop <- properties.split("\n")                                     // <4>
  if ignoreRegex.findFirstIn(prop) == None                           // <5>
  kvRegex(key, value) = prop                                         // <6>
} yield (key.trim, value.trim)                                       // <7>
// Returns: kvPairs: Array[(String, String)] = Array( 
//   (book.name,Programming Scala, Second Edition), 
//   (book.authors,Dean Wampler and Alex Payne), 
//   (book.publisher,O'Reilly), 
//   (book.publication-year,2014))
```
➊ 正则表达式用于查找将被“忽略”掉的行,例如:空行或注释行。表达式中的 # 是注释符,只有当它在该行所有非空白符中位于第一位时才能匹配该表达式。  
➋ 用于匹配 key = value 对的正则表达式,该表达式可以处理包含任意多的空白字符以及注释的情况。  
➌ 一个输入示例,该示例中包含了多行属性字符串。请留意,为了能够移除 | 字符以及该字符之前的所有的行首字符,代码中使用 StringLike.stripMargin 方法。运用这项技术,我们可以将各行缩进对齐,而且无需担心这些空白字符会作为字符串的一部分被解析。  
➍ 各个属性通过换行符分隔。  
➎ 过滤各行字符串,只留下我们不希望被忽略掉的行。  
➏ 本行左侧代码运用了模式表达式;通过正则表达式从有效的属性行中抽取出键及值对应的字符串。  
➐ 生成最终的键值对并删掉剩余无用的空白字符。  

由 于 生 成 器 调 用 了 返 回 Array 对 象 的 String.split 方 法, 因 此 上 述 示 例 返 回 了Array[(String, String)] 类型的对象。

请查看 Scala 语言规范(http://www.scala-lang.org/docu/files/ScalaReference.pdf )的 6.19 节,该节中列举了更多关于 for 推导式及其转化原理的相关示例。

### Option 以及其他的一些容器类型
我们在示例中使用了 List 、 Array 以及 Map 容器,不过除了这些明显的容器类型之外, for推导式中还可以使用任何一种实现 foreach 、 map 、 flat 以及 withFilter 方法的类型。换言之,任何提供了这些方法的类型均可视为容器,而我们也可以在 for 推导式中使用这些类型的实例。

我们将学习一些其他类型的容器。了解对这些容器应用 for 推导式会对代码造成多么难以执行的改变。

#### Option 容器
Option 是一个二元容器,其中也许包含了一个元素,也许不包含任何元素。 Option 提供了我们所需的四个方法。

下面列出了 Option 类型中所需方法的实现代码(代码摘自 Scala 2.11 版本库源代码,一些无关的细节已省略或修改):
```scala
sealed abstract class Option[+A] { self =>                 // ➊
  ...
  def isEmpty: Boolean // Some和None类型会实现该变量。

  final def foreach[U](f: A => U): Unit =
    if (!isEmpty) f(this.get)
    
  final def map[B](f: A => B): Option[B] =
    if (isEmpty) None else Some(f(this.get))

  final def flatMap[B](f: A => Option[B]): Option[B] =
    if (isEmpty) None else f(this.get)
    
  final def filter(p: A => Boolean): Option[A] =
    if (isEmpty || p(this.get)) this else None
    
  final def withFilter(p: A => Boolean): WithFilter = new WithFilter(p)
    
  /** 为了能够遵守“不创建新容器”的约定,我们需要声明WithFilter类。
   * 尽管Option容器的最大元素数为1,创建新容器似乎也不会对性能造成多大影响。
   */
  class WithFilter(p: A => Boolean) {
    def map[B](f: A => B): Option[B] = self filter p map f     // ➋
    def flatMap[B](f: A => Option[B]): Option[B] = self filter p flatMap f
    def foreach[U](f: A => U): Unit = self filter p foreach f
    def withFilter(q: A => Boolean): WithFilter =
      new WithFilter(x => p(x) && q(x))
  }
}
```
➊ self => 表达式定义了 Option 实例的一个别名,该别名在后面出现的 WithFilter 方法中被使用。如果想了解更多信息,请查看 14.6 节。  
➋ 我们需要在封闭的 Option 实例中使用之前定义的 self 引用,而不是在 WithFilter 实例中使用。也就是说,如果我们使用 this 引用,该引用将指向 WithFilter 实例。  

final 关键字会阻止子类覆写这些方法实现。当你看到 Option 这个基类中引用了继承类时,也许会感到些许震惊。因为通常情况下,如果基类知道继承类型的所有信息,该设计会被视为不好的面向对象设计。

不过,我们可以回顾下第 2 章关于 sealed 关键字的内容。 sealed 关键字意味着 Scala 只允许在相同文件中定义该类的子类。 Option 对象要么是空对象( None ),要么是非空对象( Some )。因此,这段代码是健壮、全面(能覆盖所有的场景)、简洁而且完全合理的。

这些 Option 方法具有一个重要的特性:只有当 Option 非空时,那些方法才会使用传入的函数参数。

利用这一特性,我们能够优雅地解决一个常见的设计问题。分布式计算领域中有一个常见的模式,即将计算分解为小任务,再将这些任务分发到集群中,之后再收集这些任务的执行结果。例如:Hadoop 的 MapReduce 框架就使用了这一模式。我们希望能通过一种优雅的方式忽略任务结果为空的情况,只对非空结果进行处理。暂且会忽略那些出错的任务。

首先,假设每个任务都会返回 Option 对象,其中 None 对象代表了结果为空的返回值,而Some 对象则对非空结果进行了封装。之后,我们希望以最优雅的方式过滤出非空结果。

在下面的示例中,有一个包含了三个结果值的集合,其中每个结果值均为 Option[Int]对象:
```scala
// src/main/scala/progscala2/forcomps/for-options-seq.sc

val results: Seq[Option[Int]] = Vector(Some(10), None, Some(20))

val results2a = for {
  Some(i) <- results
} yield (2 * i)
// Returns: results2a: Seq[Int] = Vector(20, 40)
```
Some(i) <- list 语句会对 results 变量中包含的元素执行模式匹配,移除 None 元素,并抽取类型为 Some 的元素的整数值。之后,生成我们所希望得到的最终表达式。而该程序输出为 Vector(20, 40) 。

下面我们做一个练习,回顾一遍 for 推导式的转化规则。首先,我们运用第一条规则,将每一个格式为 pat <- expr 的表达式转化成一个包含 withFilter 语句的表达式:
```scala
// Translation step #1
val results2b = for {
  Some(i) <- results withFilter { 
    case Some(i) => true
    case None => false 
  }
} yield (2 * i)
// Returns: results2b: Seq[Int] = Vector(20, 40)
```
最后,我们将格式为 for { x <- y } yield (z) 的表达式转化成一个 map 调用。
```scala
// Translation step #2
val results2c = results withFilter { 
  case Some(i) => true
  case None => false 
} map {
  case Some(i) => (2 * i)
}
// Returns: results2c: Seq[Int] = Vector(20, 40)
```
实际上,这条 map 表达式会生成一条编译器警告信息:
```scala
<console>:9: warning: match may not be exhaustive.
It would fail on the following input: None
    } map {
        ^
```
如果传递给 map 方法的偏函数(partial function)未使用 None => ... 子句,这种情况通常会比较危险。但如果 map 方法处理的元素出现了 None 对象,Scala 又会抛出 MatchError 的 异 常。 不 过, 由 于 调 用withFilter 的方法中已经移掉了所有的 None 元素,运行代码时便不会出现这一错误。

现在让我们再思考另一个设计难题。这个难题并不是关于忽略各个独立任务中毫无关联的空值并组合非空值的问题。问题是在执行一组非独立的操作步骤中,我们希望在获得了一个 None 值时,能够尽快停止整个相互关联的处理过程。

None 对象存在一个局限,就是你无法知道为什么这一操作不返回任何值。原因可能是出错导致了返回 None 值。针对这一局限,我们会在本章的后续内容中解决。

我们也可以编写复杂的条件逻辑代码,每次处理一个输出并检查结果值。不过使用一个for 推导式是更好的做法。
```scala
// src/main/scala/progscala2/forcomps/for-options-good.sc

def positive(i: Int): Option[Int] = 
  if (i > 0) Some(i) else None

for {
  i1 <- positive(5)
  i2 <- positive(10 * i1)
  i3 <- positive(25 * i2)
  i4 <- positive(2  * i3)
} yield (i1 + i2 + i3 + i4)
// Returns: Option[Int] = Some(3805)

for {
  i1 <- positive(5)
  i2 <- positive(-1 * i1)              // <1>   EPIC FAIL!
  i3 <- positive(25 * i2)              // <2>
  i4 <- positive(-2 * i3)              // EPIC FAIL!
} yield (i1 + i2 + i3 + i4)
// Returns: Option[Int] = None
```
➊ 将返回 None 值,“左箭头”执行了什么操作呢?  
➋ 该行代码中引用了 i2 变量,这合理吗?  

positive 函数返回一个 Option[Int] 对象。同时,假如输入值 i 是正数, positive 也返回Some(i) 对象,否则将返回 None 对象。

请留意这两个 for 推导式中第二个和第三个表达式。这两个表达式使用了之前表达式的结果值。这些表达式似乎都认为程序将按照“正常流程”运行,因此使用从 Option[Int] 对象中抽取出的 Int 值是安全的。

我们认为第一个 for 推导式能正常执行。而第二个 for 推导式也能正常执行!一旦返回了None 值,后续的表达式将会停止运行。这是因为 map 或 flatMap 不会对这些函数字面量进行处理。

接下来我们将学习其他三个具有相同属性的容器类型: Either 、 Try 以 及 Validation 类 型。名为 Scalaz 的第三方库会对这三个类型进行定义,且该库很受欢迎。

#### Either : Option 类型的逻辑扩展
略
#### Try 类型
略
#### Scalaz 提供的 Validation 类
略

## Scala  Breaks 对象
用的也不多，直接看源码吧，代码比较简单，但是函数式编程的思想可以借鉴思考一下。

思考一下scala中break 的功能是如何实现，以及源码中object Breaks extends Breaks的作用，其实源码里面的注释已经写的很清楚了，可以看到scala和python一样，源代码的注释都是比较详细明白的。
```scala
package scala
package util.control

/** A class that can be instantiated for the break control abstraction.
 *  Example usage:
 *  {{{
 *  val mybreaks = new Breaks
 *  import mybreaks.{break, breakable}
 *
 *  breakable {
 *    for (...) {
 *      if (...) break()
 *    }
 *  }
 *  }}}
 *  Calls to break from one instantiation of `Breaks` will never
 *  target breakable objects of some other instantiation.
 */
class Breaks {

  private val breakException = new BreakControl

  /**
   * A block from which one can exit with a `break`. The `break` may be
   * executed further down in the call stack provided that it is called on the
   * exact same instance of `Breaks`.
   */
  def breakable(op: => Unit) {
    try {
      op
    } catch {
      case ex: BreakControl =>
        if (ex ne breakException) throw ex
    }
  }

  sealed trait TryBlock[T] {
    def catchBreak(onBreak: =>T): T
  }

  /**
   * This variant enables the execution of a code block in case of a `break()`:
   * {{{
   * tryBreakable {
   *   for (...) {
   *     if (...) break()
   *   }
   * } catchBreak {
   *   doCleanup()
   * }
   * }}}
   */
  def tryBreakable[T](op: =>T) = new TryBlock[T] {
    def catchBreak(onBreak: =>T) = try {
      op
    } catch {
      case ex: BreakControl =>
        if (ex ne breakException) throw ex
        onBreak
    }
  }

  /**
   * Break from dynamically closest enclosing breakable block using this exact
   * `Breaks` instance.
   *
   * @note This might be different than the statically closest enclosing block!
   */
  def break(): Nothing = { throw breakException }
}

/** An object that can be used for the break control abstraction.
 *  Example usage:
 *  {{{
 *  import Breaks.{break, breakable}
 *
 *  breakable {
 *    for (...) {
 *      if (...) break
 *    }
 *  }
 *  }}}
 */
object Breaks extends Breaks
```